In [92]:
import pandas as pd
from datetime import datetime

In [93]:
CSV_PATH = "../NammaMetro_Ridership_Dataset.csv"


# Date in YYYY-MM-DD 
# or leave blank for latest date from the dataset

CARD_DATE = "2025-04-04"

In [94]:
COLUMN_MAP = {
    'date': 'Record Date',
    'total_passengers': 'Total Smart Cards',
    'smart_cards': 'Total Smart Cards',
    'tokens': 'Total Tokens',
    'qr': 'Total QR',
    'ncmc': 'Total NCMC',
    'group_ticket': 'Group Ticket',
    'stored_value': 'Stored Value Card',
    'one_day': 'One Day Pass',
    'three_day': 'Three Day Pass',
    'five_day': 'Five Day Pass',
    'qr_namma': 'QR NammaMetro',
    'qr_whatsapp': 'QR WhatsApp',
    'qr_paytm': 'QR Paytm'
}

In [95]:
df = pd.read_csv(CSV_PATH)

if isinstance(CARD_DATE, str) and CARD_DATE != "":
    try:
        CARD_DATE = datetime.strptime(CARD_DATE, "%Y-%m-%d").strftime("%d-%m-%Y")
    except ValueError:
        print(f"Invalid date format: {CARD_DATE}")
        exit()
    row = df[df[COLUMN_MAP['date']] == CARD_DATE]
    if row.empty:
        print(f"No data found for {CARD_DATE}")
        exit()
    date_for_filename = CARD_DATE
    date_display = CARD_DATE
else:
    # Get the latest date (assume sorted or find max)
    # Convert all dates to datetime for correct max
    df['date'] = pd.to_datetime(df[COLUMN_MAP['date']], format="%d-%m-%Y")
    latest_idx = df['date'].idxmax()
    row = df.loc[latest_idx]
    date_for_filename = row[COLUMN_MAP['date']][6:] + '-' + row[COLUMN_MAP['date']][3:5] + '-' + row[COLUMN_MAP['date']][:2]
    date_display = row[COLUMN_MAP['date']]


In [96]:
data = {key: row[COLUMN_MAP[key]] for key in COLUMN_MAP if key != 'date'}
data['date'] = date_display

# Calculate total passengers
total = (int(data['smart_cards'].iloc[0]) + int(data['tokens'].iloc[0]) + 
            int(data['ncmc'].iloc[0]) + int(data['group_ticket'].iloc[0]) + int(data['qr'].iloc[0]))
data['total_passengers'] = str(total)

# Calculate percentages for the chart
smartcard_pct = int(data['smart_cards'].iloc[0]) / total * 100
tokens_pct = int(data['tokens'].iloc[0]) / total * 100
qr_pct = int(data['qr'].iloc[0]) / total * 100
ncmc_pct = int(data['ncmc'].iloc[0]) / total * 100
group_pct = int(data['group_ticket'].iloc[0]) / total * 100

# Format date for better display
display_date = data['date']

# Add commas to larger numbers for readability
def add_commas(num):
    """Improved version that handles both raw numbers and pandas Series"""
    if hasattr(num, 'iloc'):  # Check if it's a pandas Series/DataFrame
        num = num.iloc[0] if len(num) > 0 else 0
    return f"{int(num):,}"

In [97]:
datacard_html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <title>Namma Metro Ridership | {data['date']}</title>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&display=swap" rel="stylesheet">
    <style>
        :root {{
            --primary: #a32a8e;   /* Namma Metro purple */
            --secondary: #2ecc71; /* Fresh green */
            --accent: #f39c12;    /* Warm orange */ 
            --ncmc: #ff3333;      /* NCMC cherry */
            --group: #9b59b6;     /* Group ticket purple */
            --dark: #2d3748;      /* Dark slate */
            --light: #f7fafc;     /* Off white */
            --gray: #a0aec0;      /* Medium gray */
        }}
        
        * {{
            box-sizing: border-box;
            margin: 0;
            padding: 0;
        }}
        
        body {{
            font-family: 'Inter', sans-serif;
            background-color: var(--light);
            color: var(--dark);
            line-height: 1.5;
            padding: 0;
            margin: 0;
        }}
        
        .card {{
            max-width: 600px;
            margin: 20px auto 20px;
            background: white;
            border-radius: 12px;
            overflow: hidden;
            box-shadow: 0 10px 25px rgba(0,0,0,0.1);
        }}
        
        .header {{
            background: var(--primary);
            color: white;
            padding: 20px;
            position: relative;
        }}
        
        .header h1 {{
            font-size: 24px;
            font-weight: 700;
            margin: 0;
            text-align: center;
        }}
        
        .header-date {{
            font-size: 20px;
            font-weight: 500;
            margin: 10px auto 0;
            opacity: 0.9;
            text-align: center;
        }}
        
        .content {{
            padding: 25px;
        }}
        
        .total-container {{
            text-align: center;
            margin-bottom: 30px;
        }}
        
        .total-passengers {{
            font-size: 48px;
            font-weight: 700;
            color: var(--dark);
            letter-spacing: -1px;
            margin-bottom: 5px;
        }}
        
        .total-label {{
            font-size: 14px;
            font-weight: 600;
            color: var(--gray);
            text-transform: uppercase;
            letter-spacing: 1px;
        }}
        
        .payment-grid {{
            display: grid;
            grid-template-columns: repeat(2, 1fr);
            gap: 15px;
            margin-bottom: 25px;
        }}
        
        .payment-box {{
            background: #f8fafc;
            border-radius: 8px;
            padding: 15px;
            border-left: 4px solid var(--primary);
        }}
        
        .payment-box.smartcard {{
            border-left-color: var(--primary);
        }}
        
        .payment-box.tokens {{
            border-left-color: var(--secondary);
        }}
        
        .payment-box.qr {{
            border-left-color: #3182ce;
        }}
        
        .payment-value {{
            font-size: 24px;
            font-weight: 700;
            color: var(--dark);
            margin-bottom: 2px;
        }}
        
        .payment-label {{
            font-size: 13px;
            text-transform: uppercase;
            color: var(--dark);
            font-weight: 500;
        }}
        
        .breakdown {{
            margin-top: 6px;
            font-size: 12px;
            color: var(--gray);
        }}
        
        .breakdown div {{
            margin-bottom: 2px;
        }}
        
        .breakdown span {{
            color: var(--gray);
            font-weight: 500;
        }}
        
        .chart {{
            height: 24px;
            background: #edf2f7;
            border-radius: 12px;
            overflow: hidden;
            display: flex;
            margin: 25px 0;
        }}
        
        .chart-segment.smartcard {{
            background: var(--primary);
            width: {round(smartcard_pct)}%;
        }}
        
        .chart-segment.tokens {{
            background: var(--secondary);
            width: {round(tokens_pct)}%;
        }}
        
        .chart-segment.qr {{
            background: #3182ce;
            width: {round(qr_pct)}%;
        }}
        
        .chart-segment.ncmc {{
            background: var(--ncmc);
            width: {round(ncmc_pct)}%;
        }}
        
        .chart-segment.group {{
            background: var(--group);
            width: {round(group_pct)}%;
        }}
        
        .chart-legend {{
            display: flex;
            flex-wrap: wrap;
            justify-content: center;
            margin-bottom: 20px;
            gap: 15px;
        }}
        
        .legend-item {{
            display: flex;
            align-items: center;
            font-size: 12px;
            color: var(--dark);
        }}
        
        .legend-color {{
            width: 12px;
            height: 12px;
            border-radius: 3px;
            margin-right: 6px;
        }}
        
        .legend-color.smartcard {{
            background: var(--primary);
        }}
        
        .legend-color.tokens {{
            background: var(--secondary);
        }}
        
        .legend-color.qr {{
            background: #3182ce;
        }}
        
        .legend-color.ncmc {{
            background: var(--ncmc);
        }}
        
        .legend-color.group {{
            background: var(--group);
        }}

        .footer {{  
            padding-top: 5px;
            margin-top: 0px; /* Add margin from bottom of screen */
            text-align: center;
            font-size: 12px;
            color: var(--gray);
            background-color: var(--dark);
            border-top: 1px solid #edf2f7;
            position: relative;
            height: 150px; /* Give enough height for the image */
        }}
        
        .footer::after {{  
            content: "";
            position: absolute;
            bottom: 0;
            left: 0;
            right: 0;
            height: 120px; /* Height of the metro train image */
            background: url('footer-img.png') no-repeat center bottom;
            background-size: contain; /* Maintain aspect ratio but fit width */
            z-index: 1;
        }}
        
        @media (max-width: 480px) {{
            .card {{
                border-radius: 0;
                box-shadow: none;
            }}
            
            .total-passengers {{
                font-size: 40px;
            }}
            
            .payment-grid {{
                grid-template-columns: 1fr;
            }}
        }}
    </style>
</head>
<body>
    <div class="card">
        <div class="header">
            <h1>Namma Metro Ridership</h1>
            <div class="logo"></div>
        </div>
        
        <div class="content">
            <div class="date-container" style="text-align: center; margin-bottom: 20px;">
                <div class="total-label">Date</div>
                <div class="total-passengers" style="font-size: 32px;">{display_date}</div>
            </div>
            
            <div class="total-container">
                <div class="total-passengers">{add_commas(data['total_passengers'])}</div>
                <div class="total-label">Total Passengers</div>
            </div>
            
            <div class="chart">
                <div class="chart-segment smartcard"></div>
                <div class="chart-segment tokens"></div>
                <div class="chart-segment qr"></div>
                <div class="chart-segment ncmc"></div>
                <div class="chart-segment group"></div>
            </div>
            
            <div class="chart-legend">
                <div class="legend-item">
                    <div class="legend-color smartcard"></div>
                    <div>Smart Cards {smartcard_pct:.1f}%</div>
                </div>
                <div class="legend-item">
                    <div class="legend-color tokens"></div>
                    <div>Tokens {tokens_pct:.1f}%</div>
                </div>
                <div class="legend-item">
                    <div class="legend-color qr"></div>
                    <div>QR {qr_pct:.1f}%</div>
                </div>
                <div class="legend-item">
                    <div class="legend-color ncmc"></div>
                    <div>NCMC {ncmc_pct:.1f}%</div>
                </div>
                <div class="legend-item">
                    <div class="legend-color group"></div>
                    <div>Group {group_pct:.2f}%</div>
                </div>
            </div>
            
            <div class="payment-grid">
                <div class="payment-box smartcard">
                    <div class="payment-value">{add_commas(data['smart_cards'])}</div>
                    <div class="payment-label">Smart Cards</div>
                    <div class="breakdown">
                        <div>Stored Value: <span>{add_commas(data['stored_value'])}</span></div>
                        <div>One Day Pass: <span>{add_commas(data['one_day'])}</span></div>
                        <div>Three Day Pass: <span>{add_commas(data['three_day'])}</span></div>
                        <div>Five Day Pass: <span>{add_commas(data['five_day'])}</span></div>
                    </div>
                </div>
                
                <div class="payment-box tokens">
                    <div class="payment-value">{add_commas(data['tokens'])}</div>
                    <div class="payment-label">Tokens</div>
                </div>
                
                <div class="payment-box qr">
                    <div class="payment-value">{add_commas(data['qr'])}</div>
                    <div class="payment-label">QR Tickets</div>
                    <div class="breakdown">
                        <div>Namma Metro: <span>{add_commas(data['qr_namma'])}</span></div>
                        <div>WhatsApp: <span>{add_commas(data['qr_whatsapp'])}</span></div>
                        <div>Paytm: <span>{add_commas(data['qr_paytm'])}</span></div>
                    </div>
                </div>
                
                <div class="payment-box" style="border-left-color: var(--ncmc);">
                    <div class="payment-value">{add_commas(data['ncmc'])}</div>
                    <div class="payment-label">NCMC</div>
                </div>
                
                <div class="payment-box" style="border-left-color: var(--group);">
                    <div class="payment-value">{add_commas(data['group_ticket'])}</div>
                    <div class="payment-label">Group Ticket</div>
                </div>
            </div>
        </div>
        
        <div class="footer">
            Source: Bangalore Metro Rail Corporation Limited
        </div>
    </div>
</body>
</html>
"""

In [98]:
with open('ridership_datacard.html', 'w') as f:
    f.write(datacard_html)

# [ridership_datacard.html](ridership_datacard.html)